# Import Dependencies

In [4]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from Scripts import VoxNet

# Use GPU if available, has issues with these small batch sizes
# if torch.cuda.is_available(): 
#     dev = "cuda:0" 
# else: 
#     dev = "cpu"

dev = "cpu"

print(dev)
device = torch.device(dev)

cpu


# Load ModelNet40

In [2]:
# Load training data
data = np.load('Data/ModelNet40/ModelNet40Train.npz', allow_pickle=True)

# Parser cuts off anything with an underscore. This array is modified to match
labels = [
    'airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car',
    'chair', 'cone', 'cup', 'curtain', 'desk', 'door', 'dresser', 'flower',
    'glass', 'guitar', 'keyboard', 'lamp', 'laptop', 'mantel', 'monitor',
    'night', 'person', 'piano', 'plant', 'radio', 'range', 'sink',
    'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet', 'tv', 'vase',
    'wardrobe', 'xbox'
]

train_data = np.expand_dims(data['data'], axis=1)
train_labels = data['labels']

print(train_data.shape)
print(train_labels.shape)

# Convert labels to integer values so they can be converted to tensors
label_as_int = []
for i in data['labels']:
   label_as_int.append(int(labels.index(i)))

train_data = torch.utils.data.TensorDataset(torch.FloatTensor(train_data).to(device),torch.LongTensor(label_as_int).to(device))


# Load test data
data = np.load('Data/ModelNet40/ModelNet40Test.npz', allow_pickle=True)

test_data = np.expand_dims(data['data'], axis=1)
test_labels = data['labels']

print(test_data.shape)
print(test_labels.shape)

# Convert labels to integer values so they can be converted to tensors
label_as_int = []
for i in data['labels']:
   label_as_int.append(int(labels.index(i)))

test_data = torch.utils.data.TensorDataset(torch.FloatTensor(test_data).to(device),torch.LongTensor(label_as_int).to(device))

(9843, 1, 32, 32, 32)
(9843,)
(2468, 1, 32, 32, 32)
(2468,)


# Load ShapeNet

In [ ]:
# Load training data
data = np.load('Data/ShapeNet/ShapeNetTrain.npz', allow_pickle=True)

# Get a list of all possible labels in dataset
labels = ['Airplane', 'Bag', 'Cap', 'Car', 'Chair', 'Earphone', 'Guitar', 'Knife', 'Lamp', 'Laptop', 'Motorbike', 'Mug', 'Pistol', 'Rocket', 'Skateboard', 'Table']

# shapenet_labels = []
# for x in data['labels']:
#     if x not in shapenet_labels:
#         shapenet_labels.append(x)
# print(shapenet_labels)

train_data = np.expand_dims(data['data'], axis=1)
train_labels = data['labels']

print(train_data.shape)
print(train_labels.shape)

# Convert labels to integer values so they can be converted to tensors
label_as_int = []
for i in data['labels']:
   label_as_int.append(int(labels.index(i)))

train_data = torch.utils.data.TensorDataset(torch.FloatTensor(train_data).to(device),torch.LongTensor(label_as_int).to(device))


# Load test data
data = np.load('Data/ShapeNet/ShapeNetTest.npz', allow_pickle=True)

test_data = np.expand_dims(data['data'], axis=1)
test_labels = data['labels']

print(test_data.shape)
print(test_labels.shape)

# Convert labels to integer values so they can be converted to tensors
label_as_int = []
for i in data['labels']:
   label_as_int.append(int(labels.index(i)))

test_data = torch.utils.data.TensorDataset(torch.FloatTensor(test_data).to(device),torch.LongTensor(label_as_int).to(device))

# Testing VoxNet

In [5]:
# Hyperparameters
batch_size = 32
num_classes = len(labels)
print(num_classes)

learning_rate = 0.01 # Decreases by a factor of 10 every 40000 batches
momentum = 0.9
weight_decay = 0.001
decay_iter = 40000
epochs = 10

# Load data in batch sizes
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=2)

model = VoxNet.VoxNet(num_classes)
model = model.float()
model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
criterion = torch.nn.CrossEntropyLoss()

training_accuracy = []
testing_accuracy = []
batch_number = 0

for epoch in range(epochs):
    if epoch % 1 == 0:
        print("epoch:", epoch+1)
    
    for (vox_grids, vox_labels) in train_loader:
        batch_number += 1

        if batch_number % decay_iter == 0:
            print("Decreasing learning rate")
            learning_rate *= 0.1

        # print(len(vox_grids))
        if (len(vox_grids) < batch_size):
            continue

        output = model(vox_grids)
        loss = criterion(output, vox_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Test VoxNet after training
    with torch.no_grad():
        num_correct = 0
        total = 0

        for (grids, labels) in train_loader:
            if (len(grids) < batch_size):
                continue

            output = model(grids)

            _, predictions = torch.max(output, 1)

            num_correct += (predictions == labels).sum().item()
            total += labels.shape[0]
        
        accuracy = (num_correct / total) * 100.0
        training_accuracy.append(accuracy)
        print("Training accuracy:", accuracy)

        num_correct = 0
        total = 0

        for (grids, labels) in test_loader:
            if (len(grids) < batch_size):
                continue

            output = model(grids)

            _, predictions = torch.max(output, 1)

            num_correct += (predictions == labels).sum().item()
            total += labels.shape[0]
        
        accuracy = (num_correct / total) * 100.0
        testing_accuracy.append(accuracy)
        print("Testing accuracy:", accuracy)

line_one = plt.plot(range(len(training_accuracy)), training_accuracy, label='Training Accuracy')
line_two = plt.plot(range(len(testing_accuracy)), testing_accuracy, label='Testing Accuracy')
plt.legend()
plt.ylim(0,100.0)
plt.ylabel('Accuracy')
plt.xlabel('Epochs')

40
Training accuracy: 64.94299674267101
Testing accuracy: 58.685064935064936


KeyboardInterrupt: 